### Rate Limits

- 20 requests every 1 seconds 
- 100 requests every 2 minutes

### First get 10000 match histories.

- First 10 columns are the champions for blue team, red team and the last columns stands for "Blue team won"

In [1]:
import cassiopeia as cass
import getpass
import random

key = getpass.getpass("Enter Riot API key:")

Enter Riot API key:········


In [2]:
cass.set_riot_api_key(key)  # This overrides the value set in your configuration/settings.
cass.set_default_region("KR")

||start_id|end_id|
|----------|------------|------------|
| Jaewoo   | 3490000000 | 3492500000 |
| Jinyoung | 3492500001 | 3495000000 |
| Hwi      | 3495000001 | 3475000000 |
| Q        | 3475000001 | 3500000000 |

In [3]:
import pickle as pkl
import os
import pandas as pd

start_id = 3492500001
end_id = 3495000000
ids = range(start_id, end_id)

dir_name = '{}-{}'.format(start_id, end_id)
os.mkdir(os.path.join('../data/', '{}-{}'.format(start_id, end_id)))

data = []
user = os.path.realpath('.').split('/')[2]
k=0

In [ ]:
for i in ids :
    dd = []
    match = cass.get_match(i)
    try :
        assert match.version == '9.1.257.7563'
        if match.queue.value in ['RANKED_SOLO_5x5'] :
            for j in range(5) :
                dd.append(match.blue_team.participants[j].champion.name)
            for j in range(5) :
                dd.append(match.blue_team.bans[j].name)
            for j in range(5) :
                dd.append(match.red_team.participants[j].champion.name)
            for j in range(5) :
                dd.append(match.red_team.bans[j].name)
                
            dd.append(match.blue_team.win)
    except :
        pass
    data.append(dd)
    
    if len(data)==10000:
        print("Making {}.pkl.".foramt(k))
        with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
            pkl.dump(data, f)
        k+=1
        data = []

if len(data)!=0:
    with open(os.path.join('../data/', dir_name, '{}.pkl'.format(k)), 'wb') as f:
        pkl.dump(data, f)

Making call: https://ddragon.leagueoflegends.com/realms/kr.json
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000000
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000001
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000002
Making call: https://ddragon.leagueoflegends.com/cdn/9.1.1/data/ko_KR/championFull.json
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000003
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000004
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000005
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000006
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000007
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000008
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000009
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000010
Making call: https://kr.api.riotga

Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000108
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000109
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000110
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000111
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000112
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000113
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000114
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000115
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000116
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000117
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000118
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000119
Making call: https://kr.api.riotgames.com/lol/match/v4/matches/3490000120
Making call: https://kr.api.riotgames.